# Pairs vectorized

#### Uses the following information:
- parents of pairs
- pairs indices
- Usual starts,stops,counts etc.

#### Basic idea:

This involves decomposition of linear matrix indices to upper traingular indices. While an indigenous decomposition was derived during CUDA pairs implementation, here, I chose to tweak one of the standard decomposition methods, which is linked as an **SO** answer a few cells below. This has been done looking at the nature of solution expected. 


In [1]:
import numpy as np
import sys
import numba

In [2]:
NUMEVENTS = 50
AVENUMJETS = 10

numjets = np.random.poisson(AVENUMJETS, NUMEVENTS).astype(np.int)
stops = np.cumsum(numjets).astype(np.int)
starts = np.zeros_like(stops)
starts[1:] = stops[:-1]

counts = stops-starts
offsets = np.zeros(len(numjets)+1)
offsets[1:] = stops[:]

In [3]:
parents = np.empty(stops[-1], dtype=np.int)

@numba.jit()
def vectorized_search(offsets, content):
    index = np.arange(len(content), dtype=int)                     # threadIdx.x on CUDA
    below = np.zeros(len(content), dtype=int)                      # just below = 0 on CUDA
    above = np.ones(len(content), dtype=int) * (len(offsets) - 1)  # same for above
    while True:
        middle = (below + above) // 2

        change_below = offsets[middle + 1] <= index                   # which "belows" must we change?
        change_above = offsets[middle] > index                        # which "aboves"?

        if not np.bitwise_or(change_below, change_above).any():    # neither? great! we're done!
            break
        else:
            below = np.where(change_below, middle + 1, below)      # vectorized "if" statement
            above = np.where(change_above, middle - 1, above)      # this is the only branch

    return middle

In [4]:
# pairs_indices should properly be called pairs_counts
pairs_indices = np.zeros(len(numjets)+1)
pairs_indices[1:] = np.cumsum(counts*(counts+1)/2)
pairs_indices = pairs_indices.astype(np.int)

In [5]:
# pairs_contents should be called pairs_indices
pairs_contents = np.arange(pairs_indices[-1]).astype(np.int)
pairs_parents = vectorized_search(pairs_indices, pairs_contents)
pairs_parents = pairs_parents.astype(np.int)

In [6]:
left = np.empty_like(pairs_contents)
right = np.empty_like(pairs_contents)

For the formula relating linear indices to upper triangular indices,
see [here](https://stackoverflow.com/questions/27086195/linear-index-upper-triangular-matrix),[here](https://stackoverflow.com/questions/19143657/linear-indexing-in-symmetric-matrices) and [here](https://math.stackexchange.com/questions/646117/how-to-find-a-function-mapping-matrix-indices)
The formula simplified is: 
```python
i = floor(( 2*n+1 - sqrt((2n+1)*(2n+1) - 8*k ))/ 2) ;
j = k - n*i + i*(i-1)/2 ;
```
Credit goes to [@keeran Brabazon](https://stackoverflow.com/users/2839128/keeran-brabazon) for the formula. I tweaked it to fit our needs.

In [7]:
left[pairs_contents] = np.floor((pairs_contents-pairs_indices[pairs_parents[pairs_contents]])/(counts[pairs_parents[pairs_contents]]*(counts[pairs_parents[pairs_contents]]-1)/2))

In [8]:
# Keep consistency with formula
n = counts[pairs_parents[pairs_contents]]
k = pairs_contents-pairs_indices[pairs_parents[pairs_contents]]

# Add offset to the pairs_indices
left[pairs_contents] = starts[pairs_parents[pairs_contents]]+ np.floor((2*n+1 - np.sqrt((2*n+1)*(2*n+1) - 8*k))/2)
i = left[pairs_contents] - starts[pairs_parents[pairs_contents]]

# Add offset and fix the issue of lower triangular indices by adding i
right[pairs_contents] = starts[pairs_parents[pairs_contents]] + k - n*i + i*(i-1)/2 + i

In [9]:
for i in range(6):
    print("Event {}\nLeft {}\n \nRight {}\n \n".format(i,left[pairs_indices[i]:pairs_indices[i+1]],right[pairs_indices[i]:pairs_indices[i+1]]))

Event 0
Left [ 0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  1  1  1  1
  2  2  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3  3  4  4  4  4
  4  4  4  4  4  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  7  7  7  7  7
  7  8  8  8  8  8  9  9  9  9 10 10 10 11 11 12]
 
Right [ 0  1  2  3  4  5  6  7  8  9 10 11 12  1  2  3  4  5  6  7  8  9 10 11 12
  2  3  4  5  6  7  8  9 10 11 12  3  4  5  6  7  8  9 10 11 12  4  5  6  7
  8  9 10 11 12  5  6  7  8  9 10 11 12  6  7  8  9 10 11 12  7  8  9 10 11
 12  8  9 10 11 12  9 10 11 12 10 11 12 11 12 12]
 

Event 1
Left [13 13 13 13 13 13 13 13 13 13 14 14 14 14 14 14 14 14 14 15 15 15 15 15 15
 15 15 16 16 16 16 16 16 16 17 17 17 17 17 17 18 18 18 18 18 19 19 19 19 20
 20 20 21 21 22]
 
Right [13 14 15 16 17 18 19 20 21 22 14 15 16 17 18 19 20 21 22 15 16 17 18 19 20
 21 22 16 17 18 19 20 21 22 17 18 19 20 21 22 18 19 20 21 22 19 20 21 22 20
 21 22 21 22 22]
 

Event 2
Left [23 23 23 23 23 23 23 23 24 24 24 24 24 24 24